# import libraries

In [1]:
import pandas as pd
import requests
import json
import datetime
import folium
from folium import plugins
import time
import matplotlib

# Read the geo json with the geometric shape of the regions

In [2]:
# file name, for all the municipalities in the Mexican state of Nuevo León
file_geo_json_nl = 'Munxentidad19_MOD.json' # nuevo leon

# read nl
with open(file_geo_json_nl) as f:
    geo_json_nl = json.load(f)

###
    geo_json_nl is a dictionary with two keys: 'type' and 'features'.
    
        The first key, 'type' returns the string 'FeatureCollection'.  
        
        The second key, 'features' is a list with 51 elements for the municipalities of the state.   
        
            Each of the 51 elements of the list is a dictionary with keys 'type', 'properties', 'geometry'.   
            
                The first key, 'type', returns 'Feature'.
                
                The second key, 'properties', is a dictionary with the keys 'state_code', 'mun_code', 'mun_name'.
                
                The third key, 'geometry', has the coordinates.
        

In [3]:
# list of codes for municipalities
mun_code_list = []
mun_name_list = []

for municipio in geo_json_nl['features']:
    mun_code_list.append( municipio['properties']['mun_code'] )
    mun_name_list.append( municipio['properties']['mun_name'] )
    
n_mun = len(mun_code_list)

In [4]:
# adjust strings with only one digit to have two digits
for i in list(range(0,len(mun_code_list))):
    if len(mun_code_list[i])==1:
        mun_code_list[i] = '0'+mun_code_list[i]

In [5]:
print(n_mun,len(mun_name_list))
print(mun_code_list)
print(mun_name_list)

51 51
['26', '31', '41', '49', '18', '48', '39', '19', '10', '21', '45', '25', '06', '12', '47', '01', '13', '20', '03', '15', '51', '44', '46', '08', '33', '50', '43', '42', '40', '38', '37', '36', '35', '34', '32', '30', '29', '28', '27', '24', '23', '22', '17', '16', '14', '11', '09', '07', '05', '04', '02']
['Guadalupe', 'Juárez', 'Pesquería', 'Santiago', 'García', 'Santa Catarina', 'Monterrey', 'San Pedro Garza García', 'Carmen', 'Gral. Escobedo', 'Salinas Victoria', 'Gral. Zuazua', 'Apodaca', 'Ciénega de Flores', 'Hidalgo', 'Abasolo', 'China', 'Gral. Bravo', 'Los Aldamas', 'Dr. Coss', 'Villaldama', 'Sabinas Hidalgo', 'San Nicolás de los Garza', 'Bustamante', 'Linares', 'Vallecillo', 'Rayones', 'Los Ramones', 'Parás', 'Montemorelos', 'Mina', 'Mier y Noriega', 'Melchor Ocampo', 'Marín', 'Lampazos de Naranjo', 'Iturbide', 'Hualahuises', 'Higueras', 'Los Herreras', 'Gral. Zaragoza', 'Gral. Treviño', 'Gral. Terán', 'Galeana', 'Dr. González', 'Dr. Arroyo', 'Cerralvo', 'Cadereyta Jiméne

# Read from INEGI data bank

## my tokens and test urls

In [6]:
# my_token_banco = 'e328b804-6d59-b5dc-77ea-482ca3c4120c'

In [7]:
# look at
# https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000002,1002000003/en/19026/true/BISE/2.0/e328b804-6d59-b5dc-77ea-482ca3c4120c?type=json
# there, the 19 is for Nuevo León State and the 026 is for Guadalupe Municipality. I will look through all the municipalities in the text

url_Ini = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/'
indicadores = '1002000002,1002000003'

## read

In [8]:
json_mun_list = []

print('begin---',datetime.datetime.now())

#for i, mun_code in zip( list(range(0,len(mun_code_list))) , mun_code_list ):
for mun_code in mun_code_list:
    time.sleep(0.0025)
    resp_tmp = requests.get( url_Ini + indicadores + '/en/' + '190' + mun_code + '/true/BISE/2.0/e328b804-6d59-b5dc-77ea-482ca3c4120c?type=json' )
    if resp_tmp.status_code==200:
        json_tmp = json.loads(resp_tmp.content)
        json_mun_list.append(json_tmp)
    
print('end-----',datetime.datetime.now())

begin--- 2020-04-25 18:58:19.603737
end----- 2020-04-25 18:59:02.873589


In [9]:
# dataframe with Nuevo Leon State valid Municipalities
df_nl = pd.DataFrame(columns=('Municipality', 'Code', 'People', 'Males', 'Females', 'Asymmetry'))

n_valid_mun = 0;
for mun_name, mun_code, data_mun in zip( mun_name_list, mun_code_list, json_mun_list ):
    if data_mun['Series'] != None:
        df_nl.loc[n_valid_mun] = [
            mun_name,#
            mun_code,# string with two digit number
            # total
            int(float( data_mun['Series'][0]['OBSERVATIONS'][0]['OBS_VALUE'] )) + int(float( data_mun['Series'][1]['OBSERVATIONS'][0]['OBS_VALUE'] )),
            # males
            int(float( data_mun['Series'][0]['OBSERVATIONS'][0]['OBS_VALUE'] )),
            # females
            int(float( data_mun['Series'][1]['OBSERVATIONS'][0]['OBS_VALUE'] )),
            # asymmetry males - females. If negative there are more females than males
            (int(float( data_mun['Series'][0]['OBSERVATIONS'][0]['OBS_VALUE'] )) - int(float( data_mun['Series'][1]['OBSERVATIONS'][0]['OBS_VALUE'] )))/(int(float( data_mun['Series'][0]['OBSERVATIONS'][0]['OBS_VALUE'] )) + int(float( data_mun['Series'][1]['OBSERVATIONS'][0]['OBS_VALUE'] )))
        ]
        n_valid_mun = n_valid_mun+1

In [10]:
# set 'Code' as the index of the dataframe
df_nl.set_index('Code',inplace=True)

In [11]:
df_nl.shape

(51, 5)

In [12]:
df_nl.dtypes

Municipality     object
People           object
Males            object
Females          object
Asymmetry       float64
dtype: object

In [13]:
df_nl['People'].iloc[0]

678006

In [14]:
df_nl.loc[['26']]

,Municipality,People,Males,Females,Asymmetry
Code,,,,,
26,Guadalupe,678006,336731,341275,-0.006702


In [15]:
df_nl.head(n_valid_mun)

,Municipality,People,Males,Females,Asymmetry
Code,,,,,
26,Guadalupe,678006,336731,341275,-0.006702
31,Juárez,256970,129324,127646,0.006530
41,Pesquería,20843,10737,10106,0.030274
49,Santiago,40469,20341,20128,0.005263
18,García,143668,72640,71028,0.011220
48,Santa Catarina,268955,134388,134567,-0.000666
39,Monterrey,1135550,561656,573894,-0.010777
19,San Pedro Garza García,122659,57622,65037,-0.060452
10,Carmen,16092,8175,7917,0.016033


# Colors

In [16]:
# this uses 
# see https://www.codespeedy.com/convert-rgb-to-hex-color-code-in-python/
# import matplotlib

In [17]:
# color function red for -max of the abs of the values, blue for +max of the abs of the values
def color_rwb(x,xMax,xMin):
    xAbsMax = max( abs(xMax), abs(xMin) )
    
    if 0<x:
        xCol = matplotlib.colors.to_hex(
            [ 
                1-x/xAbsMax,
                1-x/xAbsMax,
                1
            ]
        )
        
    if x<0:
        xCol = matplotlib.colors.to_hex(
            [ 
                1,
                1+x/xAbsMax,
                1+x/xAbsMax
            ]
        )
        
    return xCol

In [18]:
#color_rb(0.006538315251842863,max_Asymm,min_Asymm)

In [19]:
#import branca # works

In [20]:
#import geopandas # works

# Make map

In [21]:
latCen_nl = +024.95
lonCen_nl = -100.33

# define a map
map_nl = folium.Map(location=[latCen_nl, lonCen_nl], width=300, height=400, zoom_start=6, min_zoom=6, max_zoom=14)

In [22]:
# see
# https://github.com/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb

max_Asymm = df_nl['Asymmetry'].max()
min_Asymm = df_nl['Asymmetry'].min()

# nuevo intento con la asimetría
folium.GeoJson(
    r'Munxentidad19_MOD.json',
    style_function=lambda feature: {
#        'fillColor': 'blue' if ( df_nl['Asymmetry'].loc[ feature['properties']['mun_code'] ]>0 ) else 'red',
        'fillColor': color_rwb( df_nl['Asymmetry'].loc[ feature['properties']['mun_code'] ] ,max_Asymm,min_Asymm),
        'fillOpacity': 0.6,
        'color': 'black',
        'weight': 1
    }
).add_to(map_nl)

In [23]:
map_nl